# Predicción de categorías
## Christian Cuéllar

A continuación se presentan modelos que identifican y aprenden patrones a partir de un conjunto de variables que, se asume, se relacionan con una etiqueta. La finalidad de implementar estos modelos es la de generar la mejor predicción para aquellas observaciones cuya etiqueta sea desconocida. Los modelos implementados son: regresión logística y SVMs. 

## Ingesta y preprocesamiento de datos

In [75]:
import pandas as pd
import numpy as np
import csv
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
pd.options.mode.chained_assignment = None 

In [4]:
# Lectura de los datos y separación entre datos etiquetados y no etiquetados
df = pd.read_csv("/home/christian/Documents/Miroculus/test/miroculus-test/data/Miroculus.test.csv", header = None)
etiquetados = df[df[0] != '?']
no_etiquetados = df[df[0] == '?']

In [5]:
# Restamos un uno a la etiqueta para que el rango tome los valores 0/1 (en lugar de 1/2)
etiquetados['etiqueta'] = etiquetados.loc[:,0].map(lambda x: 1 if x == "2" else 0)

In [6]:
# Dividimos en conjuntos de entrenamiento y prueba
X = etiquetados.ix[:, 1:14]
y = etiquetados['etiqueta']
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=0.75)

In [7]:
# Escalamos los datos de entrenamiento
from sklearn import preprocessing
scaleX = preprocessing.StandardScaler()

scaleX.fit(X_train)
X_train = scaleX.transform(X_train)
X_test = scaleX.transform(X_test)

In [13]:
type(Y_train)

pandas.core.series.Series

Dividimos nuevamente los datos en conjuntos de validacion y de prueba para seleccionar los mejores parámetros de cada modelo.

In [72]:
X_prueba, X_valida, Y_prueba, Y_valida = train_test_split(X_train, Y_train, train_size=0.8)

## Modelos de aprendizaje supervisado

### Regresión logística

##### Selección de la constante de regularización

In [133]:
lam = np.arange(0.1,1,0.1)

lam_i = []
precision_prueba_l = []
precision_validacion_l = []

for x in range(len(lam)):
    reglog_l = LogisticRegression(C=lam[x], penalty='l1', tol=0.01)
    reglog_l.fit(X_prueba, Y_prueba)
    y_pred_log_train_l = reglog_l.score(X_prueba, Y_prueba)
    y_pred_log_test_l = reglog_l.score(X_valida, Y_valida)
    
    lam_i.append(lam[x])
    precision_prueba_l.append(y_pred_log_train_l)
    precision_validacion_l.append(y_pred_log_test_l)
    
precision_lambda = pd.DataFrame({'Lambda' : lam_i, 'Precision conjunto de prueba' : precision_prueba_l,
                                 'Precision conjunto de validación':precision_validacion_l })
precision_lambda
    

,Lambda,Precision conjunto de prueba,Precision conjunto de validación
0,0.1,0.8375,0.761905
1,0.2,0.8625,0.904762
2,0.3,0.9000,0.904762
3,0.4,0.9000,0.904762
4,0.5,0.9125,0.857143
5,0.6,0.9125,0.857143
6,0.7,0.9125,0.857143
7,0.8,0.9125,0.809524
8,0.9,0.9125,0.809524


Conforme aumenta el valor de la Lambda, la precisión en el conjunto de validación empeora. Se selecciona Lambda = 0.4, porque con este valor se maximiza la precisión en el conjunto de validación (y tiene una de las mejores precisiones para el conjunto de prueba).

###### Modelo de regresión logística con la constante óptima

In [227]:
reglog = LogisticRegression(C=0.4, penalty='l1', tol=0.01)
reglog.fit(X_train, Y_train)

# Para calcular la precisión media que se reporta abajo
y_pred_log_train = reglog.score(X_train, Y_train)
y_pred_log_test = reglog.score(X_test, Y_test)

# Etiqueta predicha:
y_predict_ltest = reglog.predict(X_test)

# Probabilidad de pertenecer a cada clase:
y_pred_prob_ltes = reglog.predict_proba(X_test)
y_pred_prob_ltes2 = np.asarray(pd.DataFrame(y_pred_prob_ltes)[[1]])

print "Precisión en el conjunto de entrenamiento", y_pred_log_train
print "Prediccion en el conjunto de prueba:", y_pred_log_test

Precisión en el conjunto de entrenamiento 0.910891089109
Prediccion en el conjunto de prueba: 0.823529411765


In [131]:
# Matriz de confusión para el test
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test,y_predict_ltest)

array([[14,  3],
       [ 3, 14]])

### SVM

##### Selección del parámetro de regularización

In [149]:
lam_svm = np.arange(0.01,0.5,0.05)
lam_j = []
precision_prueba_svm = []
precision_validacion_svm = []

for x in range(len(lam_svm)):
    svm_l = SVC(C=lam_svm[x], kernel='linear', probability= True)
    svm_l.fit(X_prueba, Y_prueba)
    y_pred_svm_train_l = svm_l.score(X_prueba, Y_prueba)
    y_pred_svm_test_l = svm_l.score(X_valida, Y_valida)
    
    lam_j.append(lam_svm[x])
    precision_prueba_svm.append(y_pred_svm_train_l)
    precision_validacion_svm.append(y_pred_svm_test_l)
#   print "Lambda =", lam[x], " Precisión en el conjunto de prueba:",  y_pred_log_train_l
#   print "Lambda =", lam[x], " Precisión en el conjunto de validación:", y_pred_log_test_l
    
precision_lambda_svm = pd.DataFrame({'Lambda' : lam_j, 'Precision conjunto de prueba' : precision_prueba_svm,
                                 'Precision conjunto de validación':precision_validacion_svm })
precision_lambda_svm


,Lambda,Precision conjunto de prueba,Precision conjunto de validación
0,0.01,0.8875,0.857143
1,0.06,0.8875,0.904762
2,0.11,0.8875,0.857143
3,0.16,0.9000,0.857143
4,0.21,0.8875,0.809524
5,0.26,0.8875,0.809524
6,0.31,0.8875,0.809524
7,0.36,0.9000,0.809524
8,0.41,0.9125,0.809524
9,0.46,0.9000,0.809524


Se elige Lambda = 0.06 porque es el valor que maximiza la precisión en el conjunto de validación

##### Modelo SVM con el parámetro de penalización óptimo

In [150]:
from sklearn.svm import SVC
svm_t = SVC(C=0.06, kernel='linear', probability= True)
svm_t.fit(X_train, Y_train) 

SVC(C=0.06, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [152]:
# Probabilidades predichas para el conjunto de prueba
Y_predict_t =  svm_t.predict_proba(X_test)
y_pred_prob_svmt = np.asarray(pd.DataFrame(Y_predict_t)[[1]])

# Precisión en ambos conjuntos
svmt_score = svm_t.score(X_train, Y_train)
svmt_score2 = svm_t.score(X_test, Y_test)

print "Predicción en el conjunto de entrenamiento", svmt_score
print "Predicción el conjunto de prueba", svmt_score2

Predicción en el conjunto de entrenamiento 0.891089108911
Predicción el conjunto de prueba 0.852941176471


La precisión en el conjunto de entrenamiento es igual que en el caso con Logit, mientras que la precisión del conjunto de entrenamiento es peor para SVM.

### PCA y Logit

Se probó reducir la dimensionalidad utilizando las primeras tres componentes principales de los datos. Las variables transformadas se utilizaron en un modelo Logit sin constante de regularización para ver si mejoraba la predicción. Los resultados se muestran a continuación.

In [67]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca.fit(X_train)

# Varianza explicada
print(pca.explained_variance_ratio_)

pca_data_train = pd.DataFrame(pca.fit_transform(X_train), columns=['primera','segunda', 'tercera'])

[ 0.24193438  0.13312597  0.09603896]


In [68]:
# Aplicar componentes principales para el test set (utilizando los resultados del train set)
pca_data_test = pd.DataFrame(pca.transform(X_test), columns=['primera','segunda', 'tercera'])

In [154]:
reglog_pca = LogisticRegression(C=1, penalty='l1', tol=0.01)
reglog_pca.fit(pca_data_train, Y_train)

# Para calcular la precisión media que se reporta abajo
y_pred_log_train_pca = reglog_pca.score(pca_data_train, Y_train)
y_pred_log_test_pca = reglog_pca.score(pca_data_test, Y_test)

# Etiqueta predicha:
y_predict_ltest_pca = reglog_pca.predict(pca_data_test)

# Probabilidad de pertenecer a cada clase:
y_pred_prob_ltes_pca = reglog_pca.predict_proba(pca_data_test)
y_pred_prob_ltes2_pca = np.asarray(pd.DataFrame(y_pred_prob_ltes_pca)[[1]])

print "Predicción en el train", y_pred_log_train_pca
print "Prediccion en el test:", y_pred_log_test_pca

Predicción en el train 0.851485148515
Prediccion en el test: 0.823529411765


## Selección del mejor modelo

Los resultados muestran que todos los modelos fueron relativamente buenos. Se elige el modelo de regresión logística con Lambda = 0.4 como el mejor porque indica la mejor precisión tanto para el conjunto de entrenamiento como para el de prueba.

## Validación cruzada

Sacamos la precisión promedio con base en la selección del mejor modelo para ver que tan robusto es

In [192]:
X1 = X.as_matrix()
y1 = y.as_matrix()

In [212]:
from sklearn import cross_validation

logit = LogisticRegression(C=0.4, penalty='l1', tol=0.01)
k_fold = cross_validation.KFold(len(X1), n_folds=10)

precision_vc = []

for k, (train, test) in enumerate(k_fold):
    logit.fit(X1[train], y1[train])
    z = logit.score(X1[test], y1[test])
    precision_vc.append(z)
    print "K =", k, " Precisión en el conjunto de prueba:",  z

print "Precisión promedio:", sum(precision_vc)/len(precision_vc)

K = 0  Precisión en el conjunto de prueba: 0.785714285714
K = 1  Precisión en el conjunto de prueba: 0.785714285714
K = 2  Precisión en el conjunto de prueba: 0.928571428571
K = 3  Precisión en el conjunto de prueba: 0.857142857143
K = 4  Precisión en el conjunto de prueba: 0.928571428571
K = 5  Precisión en el conjunto de prueba: 0.923076923077
K = 6  Precisión en el conjunto de prueba: 0.923076923077
K = 7  Precisión en el conjunto de prueba: 0.846153846154
K = 8  Precisión en el conjunto de prueba: 0.923076923077
K = 9  Precisión en el conjunto de prueba: 0.769230769231
Precisión promedio: 0.867032967033


Resulta que el modelo seleccionado es suficientemente bueno para categorizar las observaciones con independencia de los datos observados. 

## Predicción para las observaciones sin etiquetas

Por último, se generaran las predicciones para las observaciones en las que no se tiene información de la etiqueta. Se presentan tanto la probabilidad de pertenencia a una categoría, como la etiqueta predicha.

In [221]:
X_n = no_etiquetados.ix[:,1:14]
X_matrix = X_n.as_matrix()

In [238]:
# Etiqueta predicha:
prediccion_etiqueta = reglog.predict(X_matrix)

# Probabilidad de pertenecer a cada clase:
proba_etiqueta = reglog.predict_proba(X_matrix)

In [250]:
no_etiquetados['etiqueta_predicha'] = prediccion_etiqueta.tolist()
no_etiquetados['proba_etiqueta_0'] = proba_etiqueta[:,0].tolist()
no_etiquetados['proba_etiqueta_1'] = proba_etiqueta[:,1].tolist()

In [256]:
no_etiquetados.to_csv('/home/christian/Documents/Miroculus/test/miroculus-test/predictions/prediccion_no_etiquetados.csv', sep=',')

#### Predicciones para todas las observaciones con las etiquetas originales (1 y 2)

En este paso, sólo se hace la predicción para todas las observaciones utilizando la base de datos original.

In [262]:
todas = df.ix[:,1:14].as_matrix()

# Etiqueta predicha:
prediccion_etiqueta_todas = reglog.predict(todas)

# Probabilidad de pertenecer a cada clase:
proba_etiqueta_todas = reglog.predict_proba(todas)

In [264]:
prediccion_etiqueta_todas = prediccion_etiqueta_todas +1

In [267]:
df['etiqueta_predicha'] = prediccion_etiqueta_todas.tolist()
df['proba_etiqueta_1'] = proba_etiqueta_todas[:,0].tolist()
df['proba_etiqueta_2'] = proba_etiqueta_todas[:,1].tolist()

In [268]:
df.to_csv('/home/christian/Documents/Miroculus/test/miroculus-test/predictions/prediccion_todas.csv', sep=',')